In [117]:
# cofing: utf-8
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [118]:
# データの前処理
import os
import sklearn
import sklearn.model_selection
from tensorflow.keras.utils import to_categorical
batch_size = -1 # データの大きさ reshapeしてからshapeの形求める感じでやろう
timesteps = 256
input_dim = 3 # acc (x, y, z)
num_classes = -1 # 分類数

x_train, y_train, x_test, y_test = (1, 1, 1, 1)


In [119]:
# データの読み込み
home_dir = os.path.expanduser("~")
label_path = home_dir + "/Desktop/hasc_data/y_large3.csv"
data_path  = home_dir + "/Desktop/hasc_data/x_large3.csv"

label_df = pd.read_csv(label_path, index_col=0)
data_np  = np.loadtxt (data_path , delimiter=",").reshape(-1, timesteps, input_dim)

In [147]:
# label の取り扱い
labels = label_df["act"]

6 classies validation


In [159]:
# dデータ分割
y_train, y_test = sklearn.model_selection.train_test_split(labels, test_size = 0.3, train_size=None, stratify=labels)

x_train = data_np[y_train.index]
x_test  = data_np[y_test.index]
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)

num_classes = len(y_train[0])
print(num_classes, "classies validation")
batch_size = x_test.shape[0]

7 classies validation


In [160]:
input_layer = Input( shape=( timesteps, input_dim ) )
lstm1 = LSTM(units=256, activation="relu", kernel_initializer="he_normal", return_sequences=True, dropout=0.1)( input_layer )
lstm2 = LSTM(units=256, activation="relu", kernel_initializer="he_normal", return_sequences=True, dropout=0.1)( lstm1 )
lstm3 = LSTM(units=256, activation="relu", kernel_initializer="he_normal", dropout=0.1)( lstm2 )
dense = Dense(units=num_classes, activation="softmax")( lstm3 )

model = Model(inputs = input_layer, outputs = dense)
model.compile(loss="categorical_crossentropy", optimizer="Adam", metrics=["accuracy"])
model.summary()

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 256, 3)]          0         
_________________________________________________________________
lstm_47 (LSTM)               (None, 256, 256)          266240    
_________________________________________________________________
lstm_48 (LSTM)               (None, 256, 256)          525312    
_________________________________________________________________
lstm_49 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dense_15 (Dense)             (None, 7)                 1799      
Total params: 1,318,663
Trainable params: 1,318,663
Non-trainable params: 0
_________________________________________________________________


In [134]:
"""
model_to_dotがうまく動かないのでこのセルは実行しないこと

from IPython.display import SVG
from tensorflow.python.keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog="dot", format="svg"))
"""

pass

In [165]:
model.fit(x_train, y_train, batch_size=64, epochs=50, verbose=1, callbacks=[EarlyStopping(verbose=1)])

Epoch 1/50
